In [1]:
def emergePlot(binnedXr, climatXr):

    # plot indeces for multi-experiment mean, and add some lines for CI
    
    # multi-experiment mean state (by index and time)
    meanXr=climatXr.mean()
    
    dims=list(climatXr.coords)
    dims.remove('year') #this is normall experiment (for CESM) or model (for CMIP)
    
    stdXr=climatXr.mean('year').std(dims)
    
    #Z = 1.645 #Z-score for a 90% CI
    Z=1.96 #Z-score for a 95% CI

    #these are the variable/index names to plot
    #all variables that not empty
    indexToPlot = list(binnedXr.where(binnedXr.isnull().all()==False).data_vars)
    
    nIndices=len(indexToPlot)

    # define somewhere to plot
    fig = plt.figure(figsize=(18,3*nIndices))

    # for each variable number
    for i in range(nIndices):

        #create a subplot (row, column, size)
        fig.add_subplot(nIndices,1,i+1)

        # plot the actual data
        plotData=binnedXr[indexToPlot[i]]

        #blue line for multi-experiment mean, and light blue shading between min and max
        plt.plot(plotData.year, 
                 plotData.mean(dims).values, 
                 label=indexToPlot[i])
        plt.fill_between(plotData.year, 
                         plotData.min(dims).values, plotData.max(dims).values, 
                         color='lightblue')


        # draw some horizontal lines for criteria
        plt.axhline(y=meanXr[indexToPlot[i]]+Z*stdXr[indexToPlot[i]], dashes=(5,5), color='black')
        plt.axhline(y=meanXr[indexToPlot[i]]-Z*stdXr[indexToPlot[i]], dashes=(5,5), color='black')

        plt.axvline(x=1995)
        
        #Tidy up a bit
        plt.xlabel('year')
        plt.ylabel('number of events/30 years')
        #plt.xlim(1000,2100)
        #plt.ylim(0,20)
        plt.minorticks_on()
        plt.legend()
